In [12]:
!pip install pytubefix
!pip install ffmpeg-python
!pip install faster-whisper
!pip install googletrans==4.0.0rc1

In [13]:
import os
import pytubefix as pytube
url = "https://www.youtube.com/watch?v=DQacCB9tDaw"
yt = pytube.YouTube(url)
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download()
os.rename(yt.title + ".mp4", yt.title)

In [14]:
import time
import math
import ffmpeg

def extract_audio(input_file):
  extract_audio = f"audio-{input_file}.wav"
  stream = ffmpeg.input(input_file)
  stream = ffmpeg.output(stream, extract_audio)
  ffmpeg.run(stream , overwrite_output=True)
  return extract_audio
audio_extract = extract_audio(yt.title)

In [15]:
from faster_whisper import WhisperModel
def transcribe(audio):

    model = WhisperModel("small")
    segments, info = model.transcribe(audio)
    source_language = info[0]
    print(f" Transcription Language: {source_language}")
    segments = list(segments) # this is where the transcribe happens

    for segment in segments:
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    return source_language, segments

In [16]:
%%time
language, segments = transcribe(audio_extract)

 Transcription Language: en
[0.00s -> 20.96s]  Hi everyone, thank you, thank you, it's great to have you here today.
[20.96s -> 26.88s]  Today I'm going to talk about three things, that's it, we will start with why it's so
[26.88s -> 32.80s]  important to us to have a product that we can make freely available and broadly available to
[32.80s -> 39.52s]  everyone and we're always trying to find out ways to reduce friction so everyone can use
[39.52s -> 46.40s]  Chagivity wherever they are. So today we'll be releasing the desktop version of Chagivity
[46.40s -> 54.56s]  and the refreshed UI that makes it simpler to use much more natural as well. But the big news today
[54.56s -> 62.80s]  is that we are launching our new flagship model and we are calling it GPT4O. The special thing
[62.80s -> 70.88s]  about GPT4O is that it brings GPT4 level intelligence to everyone including our free users.
[71.84s -> 77.68s]  We'll be showing some live demos today to show the full extent of the capabili

In [17]:
print (language)
text_only = [segment.text for segment in segments]
for text in text_only:
  print(text)

en
 Hi everyone, thank you, thank you, it's great to have you here today.
 Today I'm going to talk about three things, that's it, we will start with why it's so
 important to us to have a product that we can make freely available and broadly available to
 everyone and we're always trying to find out ways to reduce friction so everyone can use
 Chagivity wherever they are. So today we'll be releasing the desktop version of Chagivity
 and the refreshed UI that makes it simpler to use much more natural as well. But the big news today
 is that we are launching our new flagship model and we are calling it GPT4O. The special thing
 about GPT4O is that it brings GPT4 level intelligence to everyone including our free users.
 We'll be showing some live demos today to show the full extent of the capabilities of our new
 model and we'll be rolling them out iteratively over the next few weeks. All right so let's get
 started. A very important part of our mission is to be able to make our advanced 

In [ ]:
import googletrans
from googletrans import Translator
import re
# Monkey-patch googletrans.gtoken to use updated regex

googletrans.gtoken.RE_TKK = re.compile(r'tkk:\'(.+?)\'', re.DOTALL)

translator = Translator()
translated_text = []

# normal sending text without chunking

for text in text_only:
    translated = translator.translate(text, src=language, dest='hi')
    if translated.text:
        translated_text.append(translated.text)

print(type(translated_text))
for trans in translated_text:
    print(trans)


#sedning text in predetermined chunks
# words_per_chunk = 10
# for i in range(0, len(text_only), words_per_chunk):
#     text_chunk = ' '.join(text_only[i: i + words_per_chunk])
#     translated = translator.translate(text_chunk, src=language, dest='hi')
#     if translated.text:
#         translated_text.append(translated.text)

# print(type(translated_text))
# for trans in translated_text:
#     print(trans)


#Trying to keep the segmented format


In [18]:
import googletrans
from googletrans import Translator
import re

# Monkey-patch googletrans.gtoken to use updated regex
googletrans.gtoken.RE_TKK = re.compile(r'tkk:\'(.+?)\'', re.DOTALL)

translator = Translator()
translated_text = []
texts_per_chunk = 10

def chunk_texts(texts, chunk_size):
    return [texts[i:i + chunk_size] for i in range(0, len(texts), chunk_size)]

text_chunks = chunk_texts(text_only, texts_per_chunk)

for chunk in text_chunks:
    combined_text = ' '.join(chunk)
    translated = translator.translate(combined_text, src=language, dest='hi')
    translated_chunks = translated.text.split(' ')

    index = 0
    for text in chunk:
        num_words = len(text.split())
        translated_text.append(' '.join(translated_chunks[index:index + num_words]))
        index += num_words

print(type(translated_text))
for trans in translated_text:
    print(trans)


<class 'list'>
हाय सब लोग, धन्यवाद, धन्यवाद, आज आपको यहां आना बहुत अच्छा है।आज मैं
तीन चीजों के बारे में बात करने जा रहा हूं, यह बात है, हम शुरू करेंगे कि
हमारे लिए ऐसा उत्पाद क्यों महत्वपूर्ण है जिसे हम स्वतंत्र रूप से उपलब्ध करा सकते हैं और
व्यापक रूप से सभी के लिए उपलब्ध हैं और हम हमेशा से पता लगाने की कोशिश
कर रहे हैंघर्षण को कम करें ताकि हर कोई चैगिविटी का उपयोग कर सके
जहां भी वे हैं।तो आज हम चैगिटी और ताज़ा यूआई के डेस्कटॉप संस्करण को जारी करेंगे जो इसे और अधिक
प्राकृतिक का उपयोग करने के लिए सरल बनाता है।लेकिन आज बड़ी खबर यह है कि हम अपना नया
फ्लैगशिप मॉडल लॉन्च कर रहे हैं और हम इसे GPT4O कह रहे हैं।GPT4O के बारे
में विशेष बात यह है कि यह हमारे मुक्त उपयोगकर्ताओं सहित सभी के लिए GPT4 स्तर की खुफिया
जानकारी लाता है।हम अपने नए मॉडल की क्षमताओं की पूरी सीमा दिखाने के लिए आज कुछ लाइव डेमो
शुरू कर दिया।हमारे मिशन का एक बहुत ही महत्वपूर्ण हिस्सा हमारे उन्नत एआई उपकरण को मुफ्त में सभी के
लिए उपलब्ध कराने में सक्षम होना है।हमें लगता है कि यह बहुत महत्वपूर्ण है कि लोगों
को इस बात के लिए ए